In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
df1 = pdr.get_data_yahoo('AAPL', '2011-01-01', '2021-10-31') #اقرأ الييانات "ابل" من تاريخ 1-1-2011 الى تاريخ 31-10-2021
df1 = df1[['Open', 'Close']] # اختار عمودين بس اللي هم open , close
df1 = df1.rename(columns = {'Open': 'Open_AAPL', 'Close': 'Close_AAPL'}) # اعيد تسمية العمودين 
df2 = pdr.get_data_yahoo('MSFT', '2011-01-01', '2021-10-31') #اقرأ الييانات "مايكروسوفت" من تاريخ 1-1-2011 الى تاريخ 31-10-2021
df2 = df2[['Open', 'Close']]
df2 = df2.rename(columns = {'Open': 'Open_MSFT', 'Close': 'Close_MSFT'})
df = df1.merge(df2, on = 'Date') #دمجت البيانات مع بعض عن طريق merge
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT
Date,,,,
2010-12-31,11.533929,11.520000,27.799999,27.910000
2011-01-03,11.630000,11.770357,28.049999,27.980000
2011-01-04,11.872857,11.831786,27.940001,28.090000
2011-01-05,11.769643,11.928571,27.900000,28.000000
2011-01-06,11.954286,11.918929,28.040001,28.820000
...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005
2021-10-26,149.330002,149.320007,311.000000,310.109985
2021-10-27,149.360001,148.850006,316.000000,323.170013


In [3]:
two_years = df.loc['2011-01-01': '2012-12-31'] # اخترت المدخلات او الصفوف من تاريخ 1-1-2011 اللى تاريخ 31-12-2012
two_years = two_years[['Close_AAPL', 'Close_MSFT']] #بس اعرض لي عمودين اللي هم Closeof apple and close of microsoft
two_years['Return_AAPL'] = two_years['Close_AAPL'].pct_change() #التغيير اللي يصير بين كلوز كل اليوم واليوم اللي بعده , بمعنى اخر الفرق بين اي كلوزين ورا بعض , بس كنسبة مئوية مو كقيمة فعلية طرح
two_years['Return_MSFT'] = two_years['Close_MSFT'].pct_change()
two_years = two_years[['Return_AAPL', 'Return_MSFT']] # اعرض لي البانات اللي اشتغلت عليها من السطرين السابقة
two_years = two_years.dropna() # احذف اي نان
two_years

,Return_AAPL,Return_MSFT
Date,,
2011-01-04,0.005219,0.003931
2011-01-05,0.008180,-0.003204
2011-01-06,-0.000808,0.029286
2011-01-07,0.007161,-0.007634
2011-01-10,0.018833,-0.013287
...,...,...
2012-12-24,0.001617,-0.014208
2012-12-26,-0.013784,-0.007391
2012-12-27,0.004016,0.003723


In [4]:
matrix_of_return_values = np.asmatrix(two_years) # تحويل البيانات اللي في تو ييرز الى ماتريكس (صفوف تساوي قيمة اعمدتها لتسهيل اجراء العمليات الحسابية عليها) بدل شكلها الحالي 3 اعمدة وصفوف كثيرة
matrix_of_return_values = matrix_of_return_values.T # حسب مقلوبها عشان يجري عليها العمليات الحسابية T resymbol Transpose

no_of_asset_classes = 2 # عدد شركاتي 2 ابل ومايكروسوفت 
no_of_portfolios = 10000 #no. of portof to test 
collection_of_random_asset_weights = [] # نسبة النقود اللي المفروض استثمرها في ابل وما يكروسوفت كلا على حده , كم اعطي ابل وكم اعطي مايكروسوفت
portfolio_means = [] # (apple wight + msft wight ) then took the avg ||  it means (reward)
portfolio_standard_deviations = [] # risk 
portfolio_sharpe_ratios = []  # reward \risk

def get_random_asset_weights(no_of_asset_classes):   # تطلع لي كم المفروض استثمر في كل شركة
    random_asset_weights = np.random.rand(no_of_asset_classes)
    random_asset_weights /= sum(random_asset_weights)
    random_asset_weights = np.asmatrix(random_asset_weights)
    return random_asset_weights 

def get_portfolio_statistics(asset_weights, matrix_of_return_values):
    asset_means = np.asmatrix(np.mean(matrix_of_return_values, axis = 1))
    asset_covariances = np.asmatrix(np.cov(matrix_of_return_values))
    portfolio_mean = asset_weights * asset_means
    portfolio_mean = portfolio_mean.item()
    portfolio_standard_deviation = np.sqrt(asset_weights * asset_covariances * asset_weights.T)
    portfolio_standard_deviation = portfolio_standard_deviation.item()
    portfolio_sharpe_ratio = portfolio_mean / portfolio_standard_deviation
    return portfolio_mean, portfolio_standard_deviation, portfolio_sharpe_ratio
while len(collection_of_random_asset_weights) < no_of_portfolios:
    random_asset_weights = get_random_asset_weights(no_of_asset_classes)
    collection_of_random_asset_weights.append(random_asset_weights) 
    portfolio_means.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[0])
    portfolio_standard_deviations.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[1])
    portfolio_sharpe_ratios.append(get_portfolio_statistics(random_asset_weights, matrix_of_return_values)[2])
    
index = portfolio_sharpe_ratios.index(max(portfolio_sharpe_ratios))
print(collection_of_random_asset_weights[index])

[[9.99987207e-01 1.27934553e-05]]


In [5]:
# one scenario mr Brent made he will put 75% of his money in apple and 25% in microsoft
# def get_random_asset_weights(no_of_asset_classes): هنا هو اختار الويجت بنفسه بينما نقدر نستخدم 
# اللي عرفناها في سيل 4 قبل 
some_weights = pd.DataFrame({"AAPL": [0.75], "MSFT": [0.25]}, index = [two_years.last_valid_index()]) #3معناته خلي الاندكس هنا بتاريخ اخر يوم استثمرت فيه بنهاية التو ييرز , السنتين اللي عرفتها قبل في خلية رقم 
some_weights
# ملاحظة هذه ماتريكس الويجت او الوزن اي النسبة المئوية اللي استثمر فيها مو مبلغ النقود او الفلوس الفعلية 

,AAPL,MSFT
2012-12-31,0.75,0.25


In [6]:
# if I have 10000 how much each will be based of previous percentages
capital = 10000 #SAR or $
allocations = capital*some_weights
allocations
# هنا انا اعرف الفلووس الفعلية اللي احطها في الاسهم 

,AAPL,MSFT
2012-12-31,7500.0,2500.0


In [7]:
# looking for how many shares we're going to buy with the 75000 Apple and 25000 microsoft
print(df['Close_AAPL'].loc[some_weights.last_valid_index()]) # the stock prise
print(df['Close_MSFT'].loc[some_weights.last_valid_index()])
#remember, a data frame is just an organizational tool for data. by copying no of shares from allocations, we are copying its organization - 
#then replacing the data to be what we want. he is copying the organizational work he already did :)
no_of_shares = allocations
no_of_shares['AAPL'].iloc[0] /= df['Close_AAPL'].loc[some_weights.last_valid_index()]
no_of_shares['MSFT'].iloc[0] /= df['Close_MSFT'].loc[some_weights.last_valid_index()]

#the for loop in there is doing exactly that (just in one line instead of typing it out and doing it twice)
#no_of_shares = allocations / df[[f'Close_{sym}' for sym in some_weights.columns]].loc[some_weights.last_valid_index()].values
no_of_shares

#كل اللي فات كان برمجة محفظة استثمارية واحدة عدد معين من الاسهم 

19.006071090698242
26.709999084472656


,AAPL,MSFT
2012-12-31,394.610752,93.597907


In [8]:
#I like to think of etfs like this: Sometimes when I'm hungry,
#I want Panda Express, Taco Bell, and McDonalds. 
#I wish there was a way to get just a little bit of each. 
#Buying an ETF allows you to buy into a lot of different stocks at once so I don't have to choose only one resturaunt
# we can share the risk too this way

In [9]:
#هنا انسى انو الموضوع موضوع محفظة استثمارية خاصة في واعاملها معاملة الفند (ما اعرف يرمز لايش بالعربي) لكن معناه انو انا اخذ فلوس الناس واستثمر فيها 
eight_years = df.loc['2012-12-31':] #اللي فهمته انو فنده صالح ل8 سنين من التاريخ المذكور , فاطبع كل الصفوف من تاريخ 31-12-2012 الى النهاية الصفوف
eight_years['My_Fund_Open'] = eight_years['Open_AAPL']*no_of_shares['AAPL'].iloc[0] + eight_years['Open_MSFT']*no_of_shares['MSFT'].iloc[0] 
# كم عدد الاسهم اللي املكها يوم ما كان اوبن ابل بهذا السعر في التاريخ المذكور (الاندكس هنا هو التاريخ)
# لاحظوا هنا اكرر العملية كل يوم بيومه
eight_years['My_Fund_Close'] = eight_years['Close_AAPL']*no_of_shares['AAPL'].iloc[0] + eight_years['Close_MSFT']*no_of_shares['MSFT'].iloc[0]
# نفس الشي بس يوم ما كان تقفيلة ابل بالسعر بالتاريخ المذكور
eight_years

<ipython-input-9-bb5a5086d3b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['My_Fund_Open'] = eight_years['Open_AAPL']*no_of_shares['AAPL'].iloc[0] + eight_years['Open_MSFT']*no_of_shares['MSFT'].iloc[0]
<ipython-input-9-bb5a5086d3b9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['My_Fund_Close'] = eight_years['Close_AAPL']*no_of_shares['AAPL'].iloc[0] + eight_years['Close_MSFT']*no_of_shares['MSFT'].iloc[0]


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,My_Fund_Open,My_Fund_Close
Date,,,,,,
2012-12-31,18.233213,19.006071,26.590000,26.709999,9683.790407,10000.000000
2013-01-02,19.779285,19.608213,27.250000,27.620001,10355.661647,10322.786098
2013-01-03,19.567142,19.360714,27.629999,27.250000,10307.514887,10190.488844
2013-01-04,19.177500,18.821428,27.270000,26.740000,10120.062557,9929.945971
2013-01-07,18.642857,18.710714,26.770000,26.690001,9862.287659,9881.577228
...,...,...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784
2021-10-27,149.360001,148.850006,316.000000,323.170013,88516.000605,88985.849537


In [10]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# ناديتها عشان امل logistic regression

In [11]:
short_time = 5
long_time = 15
eight_years['Close_Short_Rolling'] = eight_years['My_Fund_Close'].rolling(window = short_time).mean() #طلع لي متوسط 5 ايام من قبل التاريخ الحالي او الصف الحالي
eight_years['Close_Long_Rolling'] = eight_years['My_Fund_Close'].rolling(window = long_time).mean() #طلع لي متوسط 15 يوم من قبل التاريخ الحالي او الصف الحالي
eight_years = eight_years.dropna()
eight_years

<ipython-input-11-2e00dc4130f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['Close_Short_Rolling'] = eight_years['My_Fund_Close'].rolling(window = short_time).mean()
<ipython-input-11-2e00dc4130f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_years['Close_Long_Rolling'] = eight_years['My_Fund_Close'].rolling(window = long_time).mean()


,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,My_Fund_Open,My_Fund_Close,Close_Short_Rolling,Close_Long_Rolling
Date,,,,,,,,
2013-01-22,18.020000,18.027500,27.299999,27.150000,9666.108705,9655.028515,9589.089545,9815.460021
2013-01-23,18.171785,18.357500,27.200001,27.610001,9716.645010,9828.305163,9675.753203,9804.013698
2013-01-24,16.428572,16.089287,27.700001,27.629999,9075.553113,8935.115641,9530.109208,9711.502335
2013-01-25,16.131786,15.710000,27.580000,27.879999,8947.206594,8808.844482,9364.891508,9619.392710
2013-01-28,15.636786,16.065357,28.010000,27.910000,8792.121445,8951.880244,9235.834809,9554.188329
...,...,...,...,...,...,...,...,...
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314,87680.815288,85365.721147
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784,87760.205037,85714.925810
2021-10-27,149.360001,148.850006,316.000000,323.170013,88516.000605,88985.849537,88022.868655,86082.701927


In [12]:
#The testing set and training set can be thought of like homework and lecture.
#The lecture is curated and led by the teacher. 
#It isn't until the student does the homework on their own to see if they're right to see if the model is performing well :)

In [13]:
X = eight_years.copy()
y = np.where(X.My_Fund_Close.shift(-1) >= X.My_Fund_Close, 1, -1)  # the actual Y
index = int(0.7*len(X)) # هنا بياناتي او صفوفي تتقسم لتريننج وتستنج
X_train = X.iloc[:index]
X_test = X.iloc[index:]
y_train = y[:index]
y_test = y[index:]
logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)
predictions = logistic.predict(X_test) #Predicted Y
print(metrics.confusion_matrix(y_test, predictions))
print(logistic.score(X_test, y_test)) # النسبة المئوية لكم مرة كان توقعي صح سواء بارتفاع او انخفاض

[[ 59 236]
 [ 57 312]]
0.5587349397590361


In [14]:
results = X_test.copy()
results['Close_Diff'] = X_test.My_Fund_Close.shift(-1) - X_test.My_Fund_Close # الفرق بين تو كلوزين ورا بعض , عشان اشوف ربحت ولا خسرت بالموجب ربحت والسالب خسرت
results['Predictions'] = predictions # توقعاتي اللي ظهرت من الموديل حقي
results['True_Label'] = y_test # قيم الداتا الحقيقة 
results['Daily_Return'] = results['Close_Diff']*results['Predictions'] # قيمة الخسارة او الربح اليومية
results['Cumulative_Return'] = results['Daily_Return'].cumsum() #رصيدي من يوم ما بديت التداول كمبلغ
results.dropna(inplace = True)
results ['My_Fund_Close']['Close_Diff']['Predictions']['True_Label']['Daily_Return']['Cumulative_Return']

,Open_AAPL,Close_AAPL,Open_MSFT,Close_MSFT,My_Fund_Open,My_Fund_Close,Close_Short_Rolling,Close_Long_Rolling,Close_Diff,Predictions,True_Label,Daily_Return,Cumulative_Return
Date,,,,,,,,,,,,,
2019-03-15,46.212502,46.529999,115.339996,115.910004,29031.532171,29210.171494,28679.124775,27993.963471,342.813164,1,1,342.813164,342.813164
2019-03-18,46.450001,47.005001,116.169998,117.570000,29202.938358,29552.984657,28947.798258,28121.972618,-139.505043,1,-1,-139.505043,203.308121
2019-03-19,47.087502,46.632500,118.089996,117.650002,29634.210822,29413.479614,29134.123673,28235.218985,148.636178,1,1,148.636178,351.944299
2019-03-20,46.557499,47.040001,117.389999,117.519997,29359.547851,29562.115793,29317.918653,28356.008567,936.377181,1,1,936.377181,1288.321480
2019-03-21,47.505001,48.772499,117.139999,120.220001,29710.042905,30498.492974,29647.448906,28551.409053,-695.261389,-1,-1,695.261389,1983.582870
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-22,149.690002,148.690002,310.399994,309.160004,88122.074028,87611.402786,87500.143903,84959.679574,-116.137471,1,-1,-116.137471,63618.807910
2021-10-25,148.679993,148.639999,309.359985,308.130005,87626.170692,87495.265314,87680.815288,85365.721147,453.660469,1,1,453.660469,64072.468379
2021-10-26,149.330002,149.320007,311.000000,310.109985,88036.173232,87948.925784,87760.205037,85714.925810,1036.923753,1,1,1036.923753,65109.392132


In [3]:
results['My_Fund_Close']['Close_Diff']['Predictions']['True_Label']['Daily_Return']['Cumulative_Return']

NameError: name 'results' is not defined